In [1]:
import pandas as pd
import numpy as np
import os
import datetime
import glob
import gc
os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q2/Past_2_Years_Sales_by_zips'

In [2]:
def recursive_file_gen(root_folder):
    for root, dirs, files in os.walk(root_folder):
        for file in files:
            yield os.path.join(root, file)

In [75]:
dfiddetail = pd.read_csv('/home/jian/Projects/Big_Lots/Loyal_members/loyalty_register_data/combined_masterids_up_to_20181229_JL.csv',
                         usecols=['customer_id_hashed','customer_zip_code'])
###

master_file_2018Q1_2019Q1=pd.read_table("/home/jian/BigLots/New_Sing_Ups_2018_Fiscal_Year/All Rewards Members 2018-02-04 - 2019-05-04.zip",
                                             compression="zip",dtype=str,sep="|",usecols=['customer_id_hashed','customer_zip_code'])
dfiddetail=master_file_2018Q1_2019Q1.append(dfiddetail).drop_duplicates("customer_id_hashed")
del master_file_2018Q1_2019Q1
###

master_file_gap=pd.read_table("/home/jian/BigLots/New_Sing_Ups_2018_Fiscal_Year/MediaStorm Rewards Master P4 2019 - no transaction info.zip",
                                             compression="zip",dtype=str,sep="|",usecols=['customer_id_hashed','customer_zip_code'])
dfiddetail=master_file_gap.append(dfiddetail).drop_duplicates("customer_id_hashed")
del master_file_gap

###
all_master_files_after_201906=list(recursive_file_gen("/home/jian/BigLots/2019_by_weeks/"))
all_master_files_after_201906=[x for x in all_master_files_after_201906 if ("aster" in x) & (".txt" in x)]
all_master_files_after_201906=[x for x in all_master_files_after_201906 if x.split("ts/2019_by_weeks/MediaStorm_")[1][:10]>"2019-06-01"]
df_master_all_later_after_201906=pd.DataFrame()
for file in all_master_files_after_201906:
    df=pd.read_table(file,dtype=str,sep="|",usecols=['customer_id_hashed','customer_zip_code'])
    df_master_all_later_after_201906=df.append(df_master_all_later_after_201906)

dfiddetail=df_master_all_later_after_201906.append(dfiddetail).drop_duplicates("customer_id_hashed")
del df_master_all_later_after_201906

####
lapsed_master=pd.read_csv('/home/jian/Projects/Big_Lots/Loyal_members/loyalty_register_data/MediaStorm_Lapsed_Reward_Member_Master_from2014-08-26to2017-02-26.zip',
                     dtype = 'str',sep = '|',usecols = ['customer_id_hashed','customer_zip_code'])

dfiddetail=dfiddetail.append(lapsed_master)
####
dfiddetail=dfiddetail.drop_duplicates("customer_id_hashed")



dfiddetail['customer_zip_code'].fillna('00000',inplace = True)
dfiddetail['customer_zip_code'] = dfiddetail['customer_zip_code'].apply(lambda x: x.split("-")[0].split(" ")[0].zfill(5)[:5])

print(len(dfiddetail.index))


gc.collect()
print("dfiddetail.shape",dfiddetail.shape)

29269940
dfiddetail.shape (29269940, 2)


In [76]:
dfiddetail.shape

(29269940, 2)

In [77]:
dfiddetail.groupby("customer_zip_code")['customer_id_hashed'].count().to_frame().reset_index().sort_values("customer_id_hashed",ascending=False)


,customer_zip_code,customer_id_hashed
0,00000,3260384
62192,92335,24199
62233,92376,18547
61855,91977,18388
54924,79936,17948
61933,92064,16054
53088,77598,15832
35007,47715,15656
62337,92503,15609
61926,92057,15558


# Sales

In [4]:
def recursive_file_gen(root_path):
    for root, dirs, files in os.walk(root_path):
        for file in files:
            yield os.path.join(root,file)

In [5]:
today=datetime.datetime.now().date()
Last_Saturday=today-datetime.timedelta(days=today.weekday()+1)
print(Last_Saturday)

Saturday_2_years_earlier=Last_Saturday-datetime.timedelta(days=52*2*7)
Saturday_2_years_earlier=Saturday_2_years_earlier-datetime.timedelta(days=Saturday_2_years_earlier.weekday()+1)
print(Saturday_2_years_earlier)

2019-07-21
2017-07-16


In [6]:
# 1 historical subclass files
all_hist_subclass_files=glob.glob("/home/jian/BigLots/hist_daily_data_subclasslevel/*.txt")
df_all_hist_subclass_files=pd.DataFrame({"file_path":all_hist_subclass_files})
df_all_hist_subclass_files['week_end_dt']=df_all_hist_subclass_files['file_path'].apply(lambda x: x.split("rmDailySales_week_ending_")[1][:10])
df_all_hist_subclass_files=df_all_hist_subclass_files[(df_all_hist_subclass_files['week_end_dt']>=str(Saturday_2_years_earlier)) & (df_all_hist_subclass_files['week_end_dt']<=str(Last_Saturday))]

df_all_hist_subclass_files_max_date=df_all_hist_subclass_files['week_end_dt'].max()

df_all_hist_subclass_files_max_date

'2018-06-09'

In [7]:
# 3 historical item files

all_hist_item_files=glob.glob("/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/*.txt")
df_all_hist_item_files=pd.DataFrame({"file_path":all_hist_item_files})
df_all_hist_item_files['week_end_dt']=df_all_hist_item_files['file_path'].apply(lambda x: x.split("MediaStormDailySalesHistory")[1][:8])
df_all_hist_item_files=df_all_hist_item_files[(df_all_hist_item_files['week_end_dt']>str(df_all_hist_subclass_files_max_date).replace("-","")) & (df_all_hist_item_files['week_end_dt']<=str(Last_Saturday).replace("-",""))]

df_all_hist_item_files_max_date=df_all_hist_item_files['week_end_dt'].max()
print("df_all_hist_item_files_max_date",df_all_hist_item_files_max_date)

df_all_hist_item_files_min_date=df_all_hist_item_files['week_end_dt'].min()
print("df_all_hist_item_files_min_date",df_all_hist_item_files_min_date)

df_all_hist_item_files_max_date 20190209
df_all_hist_item_files_min_date 20180811


In [9]:
# 2 daily subclass files
daily_subclass_files=list(recursive_file_gen("/home/jian/BigLots/2018_by_weeks/"))
daily_subclass_files=[x for x in daily_subclass_files if (("aily" in x ) & (".txt" in x))]
df_daily_subclass_files=pd.DataFrame({"file_path":daily_subclass_files})

df_daily_subclass_files['week_end_dt']=df_daily_subclass_files['file_path'].apply(lambda x: x.split("ks/MediaStorm_")[1][:10])
df_daily_subclass_files=df_daily_subclass_files[(df_daily_subclass_files['week_end_dt']>str(df_all_hist_subclass_files_max_date)) &\
                                                (df_daily_subclass_files['week_end_dt'].apply(lambda x: x.replace("-",""))<str(df_all_hist_item_files_min_date))]

df_daily_subclass_files_max_date=df_daily_subclass_files['week_end_dt'].max()
print("df_daily_subclass_files_max_date",df_daily_subclass_files_max_date)

df_daily_subclass_files_min_date=df_daily_subclass_files['week_end_dt'].min()
print("df_daily_subclass_files_min_date",df_daily_subclass_files_min_date)

df_daily_subclass_files_max_date 2018-08-04
df_daily_subclass_files_min_date 2018-06-16


In [11]:
# 4 daily item files
daily_item_files=list(recursive_file_gen("/home/jian/BigLots/2019_by_weeks/"))
daily_item_files=[x for x in daily_item_files if (("aily" in x ) & (".txt" in x))]
df_daily_item_files=pd.DataFrame({"file_path":daily_item_files})

df_daily_item_files['week_end_dt']=df_daily_item_files['file_path'].apply(lambda x: x.split("ks/MediaStorm_")[1][:10])

df_daily_item_files=df_daily_item_files[(df_daily_item_files['week_end_dt'].apply(lambda x: x.replace("-",""))>str(df_all_hist_item_files_max_date)) &\
                                                (df_daily_item_files['week_end_dt']<str(Last_Saturday))]

df_daily_item_files_max_date=df_daily_item_files['week_end_dt'].max()
print("df_daily_item_files_max_date",df_daily_item_files_max_date)

df_daily_item_files_min_date=df_daily_item_files['week_end_dt'].min()
print("df_daily_item_files_min_date",df_daily_item_files_min_date)


df_daily_item_files_max_date 2019-07-20
df_daily_item_files_min_date 2019-02-16


In [12]:
all_item_files=df_daily_item_files.append(df_all_hist_item_files).sort_values("week_end_dt")
all_subclass_files=df_all_hist_subclass_files.append(df_daily_subclass_files).sort_values("week_end_dt")


In [13]:
all_item_files.shape

(50, 2)

In [14]:
all_item_files['week_end_dt'].nunique()

50

In [15]:
all_subclass_files.shape

(55, 2)

In [16]:
all_subclass_files['week_end_dt'].nunique()

55

In [18]:
len(set(all_subclass_files['week_end_dt'].unique().tolist()+all_item_files['week_end_dt'].unique().tolist()))

105

In [22]:
pd.read_table(file,nrows=10,sep="|")

,location_id,transaction_dt,transaction_id,customer_id_hashed,class_code_id,subclass_id,subclass_transaction_units,subclass_transaction_amt
0,1103,2017-07-18,4143,af9258adab9d8e063aaa6a2bdb30f3a0c3bfab000ecebd...,36404,7,2,4.50
1,1412,2017-07-17,9711,NaN,12009,1,3,3.00
2,4478,2017-07-22,487,NaN,11004,4,1,3.50
3,5248,2017-07-17,9858,NaN,11007,5,1,3.00
4,1377,2017-07-21,6956,NaN,12005,5,1,1.50
5,1926,2017-07-16,7833,52a010e1e2fcd9d58f9a356f4778a0e232fd1c2e39d3c8...,11008,8,2,4.00
6,1016,2017-07-21,1762,4523882edabe04229ea7063066ed1d08aa51ec7c715091...,55012,1,2,27.00
7,1910,2017-07-20,7044,4384eb4aa3cfb3524797454ce9fe9b1380b9ba82f427c7...,15002,1,1,6.75
8,4340,2017-07-18,1861,NaN,15002,1,1,1.50
9,5185,2017-07-20,5042,NaN,35084,1,1,2.80


In [27]:
df_rewards_sales_by_id_subclass=pd.DataFrame()

print("all_subclass_files['file_path'].nunique()",all_subclass_files['file_path'].nunique())

i_counter=0
for file in all_subclass_files['file_path'].tolist():
    try:
        df=pd.read_table(file,dtype=str,sep="|",
                         usecols=['location_id','transaction_dt','transaction_id','customer_id_hashed','subclass_transaction_amt'])
    except:
        df=pd.read_table(file,dtype=str,sep="|",
                         usecols=['location_id','transaction_date','transaction_id','customer_id_hashed','subclass_transaction_amt'])
        df=df.rename(columns={"transaction_date":"transaction_dt"})
    df=df[pd.notnull(df['customer_id_hashed'])]
    df['subclass_transaction_amt']=df['subclass_transaction_amt'].astype(float)
    
    df_sales_by_store_id=df.groupby(["location_id","customer_id_hashed"])['subclass_transaction_amt'].sum().to_frame().reset_index()
    df_sales_by_store_id=df_sales_by_store_id.rename(columns={"subclass_transaction_amt":"sales"})
    
    df_trans_by_store_id=df[['location_id','transaction_dt','transaction_id','customer_id_hashed']].drop_duplicates()
    df_trans_by_store_id=df_trans_by_store_id.groupby(['location_id','customer_id_hashed'])['transaction_id'].count().to_frame().reset_index()
    df_trans_by_store_id=df_trans_by_store_id.rename(columns={"transaction_id":"trans"})
    
    df=pd.merge(df_sales_by_store_id,df_trans_by_store_id,on=['location_id','customer_id_hashed'],how="outer")
    df_rewards_sales_by_id_subclass=df_rewards_sales_by_id_subclass.append(df)
    
    i_counter+=1
    
    print(i_counter,datetime.datetime.now(),df_rewards_sales_by_id_subclass.shape)


all_subclass_files['file_path'].nunique() 55
1 2019-07-24 14:41:17.612823 (845810, 4)
2 2019-07-24 14:41:49.460530 (1677660, 4)
3 2019-07-24 14:42:26.079061 (2576389, 4)
4 2019-07-24 14:43:01.779995 (3448076, 4)
5 2019-07-24 14:43:40.552971 (4332779, 4)
6 2019-07-24 14:44:17.189641 (5185442, 4)
7 2019-07-24 14:44:56.049772 (6079366, 4)
8 2019-07-24 14:45:35.855305 (7011042, 4)
9 2019-07-24 14:46:10.848353 (7811712, 4)
10 2019-07-24 14:46:45.912836 (8626372, 4)
11 2019-07-24 14:47:33.277174 (9743183, 4)
12 2019-07-24 14:48:20.010480 (10729674, 4)
13 2019-07-24 14:48:56.855399 (11577544, 4)
14 2019-07-24 14:49:34.738082 (12449629, 4)
15 2019-07-24 14:50:16.295736 (13397658, 4)
16 2019-07-24 14:51:00.348865 (14420795, 4)
17 2019-07-24 14:51:49.120104 (15552569, 4)
18 2019-07-24 14:52:40.638326 (16746882, 4)
19 2019-07-24 14:53:48.108362 (18255588, 4)
20 2019-07-24 14:55:01.904122 (19808319, 4)
21 2019-07-24 14:56:14.185160 (21327344, 4)
22 2019-07-24 14:57:28.859391 (22863742, 4)
23 2019-

In [32]:
df_rewards_sales_by_id_subclass.head(2)

,location_id,customer_id_hashed,sales,trans
0,1,00b71d4332bf58b08045ea1941f3def587dd559fe4287e...,6.6,1
1,1,00d2a17ca9b8b48a0cc8b39051beb82850dd1b67c165e4...,1.5,1


In [33]:
df_rewards_sales_by_id_subclass=df_rewards_sales_by_id_subclass.groupby(['location_id','customer_id_hashed'])['sales','trans'].sum().reset_index()

In [35]:
df_rewards_sales_by_id_item=pd.DataFrame()

print("all_item_files['file_path'].nunique()",all_item_files['file_path'].nunique())

i_counter=0
for file in all_item_files['file_path'].tolist():
    try:
        df=pd.read_table(file,dtype=str,sep="|",
                         usecols=['location_id','transaction_dt','transaction_id','customer_id_hashed','item_transaction_amt'])
    except:
        df=pd.read_table(file,dtype=str,sep="|",
                         usecols=['location_id','transaction_date','transaction_id','customer_id_hashed','item_transaction_amt'])
        df=df.rename(columns={"transaction_date":"transaction_dt"})
    df=df[pd.notnull(df['customer_id_hashed'])]
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    
    df_sales_by_store_id=df.groupby(["location_id","customer_id_hashed"])['item_transaction_amt'].sum().to_frame().reset_index()
    df_sales_by_store_id=df_sales_by_store_id.rename(columns={"item_transaction_amt":"sales"})
    
    df_trans_by_store_id=df[['location_id','transaction_dt','transaction_id','customer_id_hashed']].drop_duplicates()
    df_trans_by_store_id=df_trans_by_store_id.groupby(['location_id','customer_id_hashed'])['transaction_id'].count().to_frame().reset_index()
    df_trans_by_store_id=df_trans_by_store_id.rename(columns={"transaction_id":"trans"})
    
    df=pd.merge(df_sales_by_store_id,df_trans_by_store_id,on=['location_id','customer_id_hashed'],how="outer")
    df_rewards_sales_by_id_item=df_rewards_sales_by_id_item.append(df)
    
    i_counter+=1
    
    print(i_counter,datetime.datetime.now(),df_rewards_sales_by_id_item.shape)


all_item_files['file_path'].nunique() 50
1 2019-07-24 15:43:26.743726 (1101864, 4)
2 2019-07-24 15:44:20.931378 (2234355, 4)
3 2019-07-24 15:45:22.378177 (3370753, 4)
4 2019-07-24 15:46:26.617839 (4524850, 4)
5 2019-07-24 15:47:31.517742 (5684357, 4)
6 2019-07-24 15:48:33.739176 (6770560, 4)
7 2019-07-24 15:49:28.886839 (7833176, 4)
8 2019-07-24 15:50:40.522352 (9162398, 4)
9 2019-07-24 15:51:50.868184 (10451912, 4)
10 2019-07-24 15:52:49.591304 (11521149, 4)
11 2019-07-24 15:53:48.678833 (12605091, 4)
12 2019-07-24 15:54:52.648935 (13750898, 4)
13 2019-07-24 15:55:52.950093 (14930027, 4)
14 2019-07-24 15:57:00.904822 (16183163, 4)
15 2019-07-24 15:58:21.902626 (17567246, 4)
16 2019-07-24 16:00:14.463715 (19413962, 4)
17 2019-07-24 16:01:59.249002 (21188482, 4)
18 2019-07-24 16:03:50.789594 (23073207, 4)
19 2019-07-24 16:05:39.400907 (24933096, 4)
20 2019-07-24 16:07:44.323729 (26990775, 4)
21 2019-07-24 16:09:12.750328 (28476967, 4)
22 2019-07-24 16:10:10.231245 (29616791, 4)
23 2019-

In [36]:
df_rewards_sales_by_id_item=df_rewards_sales_by_id_item.groupby(['location_id','customer_id_hashed'])['sales','trans'].sum().reset_index()


In [78]:
df_rewars_sales_all=df_rewards_sales_by_id_item.append(df_rewards_sales_by_id_subclass)
df_rewars_sales_all=df_rewars_sales_all.groupby(['location_id','customer_id_hashed'])['sales','trans'].sum().reset_index()
print(df_rewars_sales_all.shape)
df_rewars_sales_all.head(2)

(33495691, 4)


,location_id,customer_id_hashed,sales,trans
0,1,0000b201f4ee35c0b272fc2161b77d0e113096f4a62cad...,180.35,7
1,1,00039288d93a71887e27dd93c2209419ab0224532f2d45...,292.72,4


In [79]:
df_rewars_sales_all=pd.merge(df_rewars_sales_all,dfiddetail,on="customer_id_hashed",how="left")
df_rewars_sales_all['customer_zip_code']=df_rewars_sales_all['customer_zip_code'].fillna("00000")

In [80]:
df_rewars_sales_zip=df_rewars_sales_all.groupby(['location_id','customer_zip_code'])['sales','trans'].sum().reset_index()
df_rewars_sales_zip=df_rewars_sales_zip.sort_values(['location_id','sales','trans'],ascending=[True,False,False])
df_rewars_sales_zip.shape

(2620110, 4)

In [81]:
nielsen_zips=pd.read_excel("/home/jian/Docs/Geo_mapping/Zips by DMA by County16-17 nielsen.xlsx",skiprows=1,usecols=[0],dtype=str)
nielsen_zips.columns=['customer_zip_code']
nielsen_zips['valid_zip']="Valid_Zip"
nielsen_zips=nielsen_zips.drop_duplicates()
nielsen_zips.head(2)

,customer_zip_code,valid_zip
0,00501,Valid_Zip
1,00544,Valid_Zip


In [82]:
df_rewars_sales_zip.head(2)

,location_id,customer_zip_code,sales,trans
727,1,43232,1201454.97,33784
703,1,43207,704307.65,14150


In [83]:
df_rewars_sales_zip=pd.merge(df_rewars_sales_zip,nielsen_zips,on="customer_zip_code",how="left")
df_rewars_sales_zip['valid_zip']=df_rewars_sales_zip['valid_zip'].fillna("Unknown_Zip")

In [84]:
df_rewars_sales_zip.shape

(2620110, 5)

In [93]:
df_rewars_sales_zip_valid=df_rewars_sales_zip[df_rewars_sales_zip['valid_zip']=="Valid_Zip"]
df_rewars_sales_zip_valid.shape

(2551531, 5)

In [95]:
df_rewars_sales_zip.to_csv("./BL_sales_by_zip_2_years_up_to_"+str(Last_Saturday)+".csv",index=False)
df_rewars_sales_zip_valid.to_csv("./BL_sales_by_zip_valid_only_2_years_up_to_"+str(Last_Saturday)+".csv",index=False)


In [96]:
df_rewars_sales_zip_valid['customer_zip_code'].nunique()

38331

In [87]:
df_rewars_sales_all.shape

(33495691, 5)

In [88]:
df_rewars_sales_all.head(2)

,location_id,customer_id_hashed,sales,trans,customer_zip_code
0,1,0000b201f4ee35c0b272fc2161b77d0e113096f4a62cad...,180.35,7,92020
1,1,00039288d93a71887e27dd93c2209419ab0224532f2d45...,292.72,4,43081


In [89]:
df_rewars_sales_all['customer_id_hashed'].nunique()

23605010

In [90]:
df_rewars_sales_all.to_csv("./df_rewars_sales_all.csv",index=False)

In [91]:
df_qc_store_1=df_rewars_sales_zip[df_rewars_sales_zip['location_id']=="1"]

df_qc_store_1['sales'].sum()

5439250.4899999928

In [92]:
df_qc_store_1[df_qc_store_1['valid_zip']=="Unknown_Zip"]['sales'].sum()

371119.50999999989

In [94]:
371119.50999999989/5439250.4899999928

0.06822989871165143